In [1]:
!git clone https://github.com/h2oai/h2o-llmstudio.git
!cd h2o-llmstudio && git checkout v1.11.1 -q
!cp -r h2o-llmstudio/. ./
!rm -r h2o-llmstudio

Cloning into 'h2o-llmstudio'...
remote: Enumerating objects: 6259, done.
remote: Counting objects: 100% (1733/1733), done.
remote: Compressing objects: 100% (443/443), done.
remote: Total 6259 (delta 1551), reused 1290 (delta 1290), pack-reused 4526 (from 3)
Receiving objects: 100% (6259/6259), 56.01 MiB | 17.48 MiB/s, done.
Resolving deltas: 100% (4434/4434), done.


In [2]:
# Install python 3.10
!sudo add-apt-repository ppa:deadsnakes/ppa -y > /dev/null
!sudo apt install python3.10 python3.10-distutils psmisc -y > /dev/null
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10 > /dev/null

# install requirements
!make setup > /dev/null

Output streaming troncato alle ultime 5000 righe.
--hash=sha256:11c39fbc4280d7420684494373b7c5904fa72a2b48ef543a56c2d412999c9e5d 
--hash=sha256:11cc3afd8160d44582543838b7e4f9aa5e97865322844b75d51bf4e0e413bb3e 
--hash=sha256:1537b3dd62d8aae644f3518c407aa8469e3fd0f179cdf86c5992792713ed717a 
--hash=sha256:16ca9c90da4b1f50f089e14485db8c20cbfff2d55424062791a7392b5a9b3ff9 
--hash=sha256:176a1b524a3bd3314ed47029a86d02d5a95cc0bee15bd3063a1e1ec62b947de6 
--hash=sha256:18955c1da6fc39d957adfa346f75226246b6569e096ac9e40f67d102278c3bcb 
--hash=sha256:1bb5b50dc6dd671eb46a605a3e2eb98deb4a9af787a08fcdddabe5d824bb9664 
--hash=sha256:1c768e7584c45094dca4b334af361e43b0aaa4844c04945ac7d43379eeda9bc2 
--hash=sha256:1dd4f692304854352c3e396e9b5f0a9c9e666868dd0bdc784e2ac4c93092d87b 
--hash=sha256:25785d038281cd106c0d91a68b9930049b6464288cea59ba95b35ee37c2d23a5 
--hash=sha256:287e39ba24e141b046812c880f4619d0ca9e617235d74abc27267194fc0c7835 
--hash=sha256:2c1467d939932901a97ba4f979e8f2642415fcf02ea12f53a4e3206c

In [11]:
import pandas as pd
import csv

# Load your Excel file
df = pd.read_excel("data/llama_requests_and_responses.xlsx")

# Rename columns
df.columns = ["instruction", "output"]

# Save to CSV with proper quoting to preserve formatting
df.to_csv("data/llama_requests_and_responses_converted.csv", index=False, quoting=csv.QUOTE_ALL)

## Configurations

In H2O LLM Studio you can define various finetuning parameters using the dataclasses. Here, we will be fine-tuning the [h2oai/h2o-danube3-500m-base](https://huggingface.co/h2oai/h2o-danube3-500m-base) foundation model. Have a look at our [tooltips folder](https://github.com/h2oai/h2o-llmstudio/tree/main/documentation/docs/tooltips/experiments) to explore them in more detail.

In [12]:
%%writefile my_cfg.yaml

architecture:
    backbone_dtype: int4
    force_embedding_gradients: false
    gradient_checkpointing: true
    intermediate_dropout: 0.0
    pretrained: true
    pretrained_weights: ''
augmentation:
    neftune_noise_alpha: 0.0
    random_parent_probability: 0.0
    skip_parent_probability: 0.0
    token_mask_probability: 0.0
dataset:
    add_eos_token_to_answer: true
    add_eos_token_to_prompt: true
    add_eos_token_to_system: true
    answer_column: output
    chatbot_author: H2O.ai
    chatbot_name: h2oGPT
    data_sample: 1.0
    data_sample_choice:
    - Train
    - Validation
    limit_chained_samples: false
    mask_prompt_labels: true
    only_last_answer: false
    parent_id_column: None
    personalize: false
    prompt_column:
    - instruction
    prompt_column_separator: \n\n
    system_column: None
    text_answer_separator: <|answer|>
    text_prompt_start: <|prompt|>
    text_system_start: <|system|>
    train_dataframe: data/llama_requests_and_responses_converted.csv
    validation_dataframe: None
    validation_size: 0.1
    validation_strategy: automatic
environment:
    compile_model: false
    deepspeed_allgather_bucket_size: 1000000
    deepspeed_method: ZeRO2
    deepspeed_reduce_bucket_size: 1000000
    deepspeed_stage3_param_persistence_threshold: 1000000
    deepspeed_stage3_prefetch_bucket_size: 1000000
    find_unused_parameters: false
    gpus:
    - '0'
    huggingface_branch: main
    mixed_precision: true
    mixed_precision_dtype: bfloat16
    number_of_workers: 8
    seed: -1
    trust_remote_code: true
    use_deepspeed: false
experiment_name: my_h2o_llm_studio_experiment
llm_backbone: EleutherAI/gpt-neo-1.3B
logging:
    log_all_ranks: false
    log_step_size: absolute
    logger: None
    neptune_project: ''
    wandb_entity: ''
    wandb_project: ''
output_directory: my_h2o_llm_studio_experiment
prediction:
    batch_size_inference: 0
    do_sample: false
    max_length_inference: 256
    max_time: 0.0
    metric: Exact Match
    metric_gpt_model: gpt-3.5-turbo-0301
    metric_gpt_template: general
    min_length_inference: 2
    num_beams: 1
    num_history: 4
    repetition_penalty: 1.0
    stop_tokens: ''
    temperature: 0.0
    top_k: 0
    top_p: 1.0
problem_type: text_causal_language_modeling
tokenizer:
    add_prompt_answer_tokens: false
    max_length: 1024
    padding_quantile: 1.0
    tokenizer_kwargs: '{"use_fast": true, "add_prefix_space": false}'
training:
    attention_implementation: auto
    batch_size: 2
    differential_learning_rate: 1.0e-05
    differential_learning_rate_layers: []
    drop_last_batch: true
    epochs: 3
    evaluate_before_training: false
    evaluation_epochs: 1.0
    freeze_layers: []
    grad_accumulation: 2
    gradient_clip: 0.0
    learning_rate: 0.00005
    lora: true
    lora_alpha: 16
    lora_dropout: 0.05
    lora_r: 4
    lora_target_modules: ''
    lora_unfreeze_layers: []
    loss_function: TokenAveragedCrossEntropy
    min_learning_rate_ratio: 0.0
    optimizer: AdamW
    save_checkpoint: last
    schedule: Cosine
    train_validation_data: false
    use_dora: false
    use_rslora: false
    warmup_epochs: 0.0
    weight_decay: 0.0


Overwriting my_cfg.yaml


In [13]:
%%writefile run.sh

pipenv run python train.py -Y my_cfg.yaml &

wait
echo "all done"

Overwriting run.sh


In [14]:
!sh run.sh

[2025-05-31 12:54:23,580] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible
Keys {'force_embedding_gradients'} are not in the config.
2025-05-31 12:54:27,366 - INFO: Problem Type: text_causal_language_modeling
2025-05-31 12:54:27,367 - INFO: Global random seed: 509887
2025-05-31 12:54:27,367 - INFO:

In [15]:
import pandas as pd
val_outputs = pd.read_csv("my_h2o_llm_studio_experiment/validation_predictions.csv")

In [16]:
val_outputs.head()

,instruction,output,pred_output
0,Human: Create a ResettableCounterActor in Scal...,"import akka.actor.{Actor, ActorSystem, Props}\...","import akka.actor.{Actor, ActorSystem, Props}\..."
1,Human: Create a typed actor cluster example th...,// TypedClusterAwareRouterExample.scala\n// No...,"import akka.actor.{Actor, ActorSystem, Props}\..."
2,Human: Create a BatchingActor in Scala that ba...,"import akka.actor.{Actor, ActorSystem, Props}\...","import akka.actor.{Actor, ActorSystem, Props}\..."
3,Human: Create a SelfStoppingActor in Scala tha...,"import akka.actor.{Actor, ActorSystem, Props}\...","import akka.actor.{Actor, ActorSystem, Props}\..."
4,Human: Make a MultiResponderActor that respond...,"import akka.actor.{Actor, ActorSystem, Props}\...","import akka.actor.{Actor, ActorSystem, Props}\..."


### Inference and prompting

You can also load the trained model and manually prompt it.

In [19]:
!pipenv run python prompt.py --e my_h2o_llm_studio_experiment/

[2025-05-31 14:23:38,170] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible
Traceback (most recent call last):
  File "/content/prompt.py", line 20, in <module>
    from llm_studio.src.utils.modeling_utils import load_checkpoint, set_generation_config
  File "/content/llm_studio/src/utils/modeling_u